In [10]:
import numpy as np

import math

import import_ipynb

from tools.ToolBox import *
from tools.Extract_Experiment_Characteristic import *

In [11]:
def midline(x_list, y_list, keep_len_tot = False):
    """Function that compute:
        -> the length of each midline segment
        -> the length between the midline origin and a midline coodinate
        
    Arguments:
        -> x_list & y_list : a list of X_coordinates & a list of Y_coordinates"""
    
    # Instantiate variable
    # Note: for length_total a first value is instantiate to allow addition in the loop, might be is removed at the end
    length_total = [0]
    length_segment = []
    
    # Calculate distance between each pair of coordinates
    for i in range(len(x_list) - 1):
        segment_ = segment_length((x_list[i], y_list[i]), (x_list[i+1], y_list[i+1]))
        
        length_total.append(length_total[i]+segment_)
        length_segment.append(segment_)
    
    if keep_len_tot == True:
        # remove the first element of the list (there is only one 0 is the list)
        length_total.pop(0)
    else: 
        length_total.pop()
    
    return [length_segment, length_total]

In [12]:
def midline_reformater(x_list, y_list, nb_segment):
    """
    Function that reduce midline number of segment
    
    Arguments:
        x_list & y_list : a list of X_coordinates & a list of Y_coordinates
        nb_segment: the number of segment expected in the reformated midline
        
    Note: the result of this function goes into the function 
        
    """
    
    # Origin of the midline
    reformated_midline = []
    
    # Get total length of the midline
    _, length = np.asarray(midline(x_list, y_list))
    total_length = length[-1]
    
    # Segment the midline in segment of approximatively the same length
    for i in range(nb_segment-1):
        travelled_distance = round(total_length/nb_segment*i,3)
        midline_length = np.round(length, 3)
        
        abs_ = round(min(abs(midline_length - travelled_distance)),3)
        
        idx_neg = round(abs_ + travelled_distance,3)
        idx_pos = round(-abs_ + travelled_distance,3)
        
        try:
            idx = np.where(midline_length == idx_neg)[0][0]
        except:
            pass
            
        try: 
            idx = np.where(midline_length == idx_pos)[0][0]
        except:
            pass
        
        reformated_midline.append((x_list[idx], y_list[idx]))
    
    # Add the last Point of the midline
    reformated_midline.append((x_list[-1], y_list[-1]))
    
    
    return reformated_midline

In [13]:
def edge(x_list, y_list, keep_len_tot = False):
    """Function that compute:
        -> the length of each midline segment
        -> the length between the midline origin and a midline coodinate
        
    Arguments:
        -> x_list & y_list : a list of X_coordinates & a list of Y_coordinates"""
    
    # Instantiate variable
    # Note: for length_total a first value is instantiate to allow addition in the loop, might be is removed at the end
    length_total = [0]
    length_segment = []
    
    # Calculate distance between each pair of coordinates
    for i in range(len(x_list) - 1):
        segment_ = segment_length((x_list[i], y_list[i]), (x_list[i+1], y_list[i+1]))
        
        length_total.append(length_total[i]+segment_)
        length_segment.append(segment_)
    
    last_segment = segment_length((x_list[-1], y_list[-1]), (x_list[1], y_list[1]))
    
    length_total.append(length_total[-1]+last_segment)
    length_segment.append(last_segment)
    
    
    if keep_len_tot == True:
        # remove the first element of the list (there is only one 0 is the list)
        length_total.pop(0)
    else: 
        length_total.pop()
    
    return [length_segment, length_total]

In [14]:
def edge_reformater(x_list, y_list, nb_segment):
    """
    Function that reduce edge number of segment
    
    Arguments:
        x_list & y_list : a list of X_coordinates & a list of Y_coordinates
        nb_segment: the number of segment expected in the reformated edge
        
    """
    
    # Origin of the edge
    reformated_edge = []
    
    # Get total length of the edge
    _, length = np.asarray(edge(x_list, y_list))
    total_length = length[-1]
    
    # Segment the edge in segment of approximatively the same length
    # Note: the round(X, 3) force all the value to have 3 float to allow the use of np.where()
    if nb_segment >= len(x_list)-1:
        for idx in range(len(x_list)):
            reformated_edge.append((x_list[idx], y_list[idx]))
        
    else:
        for idx in range(nb_segment-1):
            travelled_distance = round(total_length/nb_segment*idx,3)
            edge_length = np.round(length, 3)

            abs_ = round(min(abs(edge_length - travelled_distance)),3)

            idx_neg = round(abs_ + travelled_distance,3)
            idx_pos = round(-abs_ + travelled_distance,3)

            try:
                idx = np.where(edge_length == idx_neg)[0][0]
            except:
                pass

            try: 
                idx = np.where(edge_length == idx_pos)[0][0]
            except:
                pass

            reformated_edge.append((x_list[idx], y_list[idx]))
    
    # Add the last Point of the edge
    reformated_edge.append((x_list[0], y_list[0]))
    
    return reformated_edge

In [15]:
def edge_charac(x_list, y_list, nb_segment = 9999999):
    """ Function that transform a "raw" edge in a define number of segment
    
    Arguments:
        -> x_list: a list of x_coordinate
        -> y_list: a list of y_coordinate
        -> nb_segment: the number of 'new' segment
        
    Return:
        -> x1, y1: the x/y coordinate of segment start
        -> x2, y2: the x/y coordinate of segment end
        -> slope: the slope of the segment
        -> ori: origine of the slope"""
        
    reformated_edge = edge_reformater(x_list, y_list, nb_segment)
    
    # First define the interception of the local othorgonal midline woth the edge:
    full_edge = []
    
    # Get edge segment characteristic
    for i in range(len(reformated_edge)):
        x1 = reformated_edge[i-1][0] 
        y1 = reformated_edge[i-1][1]
        
        x2 = reformated_edge[i][0]
        y2 = reformated_edge[i][1]
        
        slope_ = slope((x1, y1), (x2,y2))
        ori = origin((x1, y1), slope_)
        
        full_edge.append([x1, y1, x2, y2, slope_, ori])
    
    return full_edge

In [16]:
def sub_segment_char(angle, slope, origin, point_a, point_b, seg_len, n_sub_segment):
    """ Compute coordinate of each sub_segment
    Arguments: 
        -> angle: angle of the segment
        -> slope: slope of the segment
        -> origin: origin of the segment
        -> point_a: start point of the segment
        -> point_b: end point of the segment
        -> n_sub_segment: number of segment to consider
        
    Output:
        a list of list containing for each sub_segment:
            -> newX: X coordinate
            -> newY: Y coordinate
            -> orthogonal slope of the segment at coordinates newX/newY
            -> origin of this orthogonal line
            
    Note:
       the slope is not calculated at the mid point of the segment but at start and end -> n_sub_segment +1 values are computated""" 
    
    # Get length of sub segment
    sub_len = seg_len/n_sub_segment
    
    # instantiate output variable  
    sub_segment = []
    
    # Get all slope and origin of sub_segment in a segment     
    for i in range(n_sub_segment+1):
        newX = math.sin(math.radians(angle))*(sub_len*i) + point_a[0]
        newY = math.cos(math.radians(angle))*(sub_len*i) + point_a[1]
        ortho_slope = -1/slope
        ortho_origin = newY - ortho_slope*newX
        
        sub_segment.append((newX, newY, ortho_slope, ortho_origin))
        
    return sub_segment

In [17]:
def segment_intercept(segment_a, list_of_segment):
    """Function that compute if a segment (in a list) is intercepted by a semi-line
    
    Argument:
        -> segment_a: the semi line of interest [X_coordinate, Y_coordinate, slope, origin]
        -> list_of_segment: [x_coord start, y_coord start, x_coord end, y_coord end, slope, origin]
        
    Return a list of list containing:
        -> idx: the index of the segment intercepted in the list
        -> side: the side of segment (left mean, the segment is on top of the line origine, right mean the segment is under the line origine)
        -> length: length between the origin and the intercept
        
    Biological meaning: compute the distance between a point of the midline and the edge of the worm.
    In theory, two edge segments are intercepted.
    """
    # properties of the segment orthogonal at the midline
    x1 = segment_a[0]
    y1 = segment_a[1]
    a_m = segment_a[2]
    b_m = segment_a[3]
    
    edge_intercept = []
    
    for idx, a_segment in enumerate(list_of_segment):
        # properties of the edge segment
        x3    = a_segment[0]
        y3    = a_segment[1]
        x4    = a_segment[2]
        y4    = a_segment[3]
        a_e   = a_segment[4]
        b_e   = a_segment[5]
        
        # Coordinate of the intercept of the edge segment whith the orthogonal midline segment 
        x_intercept = (b_e - b_m)/(a_m - a_e)
        y_intercept = a_e*x_intercept + b_e
        
        # test if the intercept is inside the edge segment
        if min(x3, x4) <= x_intercept <= max(x3, x4) and min(y3, y4) <= y_intercept <= max(y3, y4):
            # Get the side of the intercept
            if y_intercept > y1:
                side = 'left'
            else:
                side = 'right'
                
            # Get the length between the midline and the edge
            length = segment_length((x1, y1), (x_intercept, y_intercept))
            
            edge_intercept.append([side, length])
                
    return edge_intercept

In [18]:
def aggregate_segment_char(x_list_midline, y_list_midline, x_list_edge, y_list_edge, n_midline_seg = 50, n_sub_segment = 25, n_edge_seg = 9999):
    """Function that aggregate all usefull data of a midline/edge
    Arguments: 
        -> x_list_midline: a list of X coordinate
        -> y_list_midline: a list of Y coordinatesment 
        -> x_list_edge: a list of X coordinate
        -> y_list_edge: a list of Y coordinate
        -> n_midline_seg: the number of segment expected in the reformated midline
        -> n_sub_segment: the number of segment inside each segment (for lateral precision)
        -> n_edge_seg: the number of segment expected in the reformated edge
        
    The function return for each segment: 
        -> an index
        -> it's origin
        -> it's end
        -> segment_length
        -> total_length
        -> it's angle
        -> it's slope
        -> sub_segment_characteristics"""
    
    midline_ = []
    tot_len = 0
    
    # Reformat the edge data of the worm
    edge_seg_list = edge_charac(x_list_edge, y_list_edge, n_edge_seg)
    
    # Reformat the midline data of the worm
    newMidline = midline_reformater(x_list_midline, y_list_midline , nb_segment = n_midline_seg)
    
    # iterate in each midline segment
    for i in range(len(newMidline)-1):
        index = i
        point_a = newMidline[i]
        point_b = newMidline[i+1]
        seg_len = segment_length(point_a, point_b)
        
        angle   = angle_from_coordinates(point_a, point_b)
        slope_origin = get_segment_slope_and_origin((point_a, point_b))
        
        # segment each midline segment in n_sub_segment
        sub_seg_list = sub_segment_char(angle, slope_origin[0][0], slope_origin[0][1], point_a, point_b, seg_len, n_sub_segment)
        
        # for each of the sub_segment, calculate the distance to the edge and its side (with midline as ref and head on right side)
        sub_seg_char = []
        for idx, sub_seg in enumerate(sub_seg_list):
            # get side (left/right) and length of the midline to edge segment
            edge_mid_side_and_distance = segment_intercept(sub_seg, edge_seg_list)
            # get length of the subsegment
            sub_seg_length = seg_len/n_sub_segment*idx
        
            # Return 
            sub_seg_char.append((sub_seg_length, edge_mid_side_and_distance))
        
        midline_.append([index, point_a, point_b, seg_len, tot_len, angle, slope_origin[0][0], slope_origin[0][1], sub_seg_char])
        
        # Put after the append to keep the first value = 0
        tot_len += seg_len
    
    return midline_

In [19]:
def midline_edge_extractor(pathExcel):
    ''' Get midline and edge characteristic from an appropriate excel file'''
    
    db = get_xls_values(pathExcel)

    x_mid = db['worm_midline']['x']
    y_mid = db['worm_midline']['y']
    x_edg = db['worm_edge']['x']
    y_edg = db['worm_edge']['y']
    
    orientation = db['worm_orientation']

    newY_mid, newY_edg = [], []
    for y in y_mid:
        newY_mid.append(-y)

    for y in y_edg:
        newY_edg.append(-y)


    worm = [x_mid, newY_mid, x_edg, newY_edg, orientation]
    
    return worm